In [16]:
import pandas as pd
import json
from module.load_json import *
from module.utils import *
from module.maps import *

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, 
    TrainingArguments, Trainer,
)

In [17]:
train_path = 'dataset/NIKL_ABSA_2021_v1.0/EXSA2112203180.json'
dev_path = 'dataset/NIKL_ABSA_2021_v1.0/EXSA2122203180.json'

In [18]:
train = jsonload(train_path)
dev = jsonload(dev_path)
train = pd.DataFrame(train['document'])
dev = pd.DataFrame(dev['document'])
total = pd.concat([train, dev]).reset_index(drop=True)

In [19]:
total.domain.unique()
categories = ['제품 기타', '전자기기', '화장품/세정제']

checker = total.domain.apply(lambda x: x in categories)
total = total[checker]
total = total[total.isna().any(axis=1) == False]
total = total[['sentence', 'opinions']].reset_index(drop=True)

In [20]:
def collect_sentences(sentences):
    collected_sentences = []
    for sentence in sentences:
        collected_sentences.append(sentence['sentence_form'])
        collected_sentences = collected_sentences[:5]
    return ' '.join(collected_sentences)

In [21]:
def collect_opinions(opinions):
    collected_opinions = []
    for opinion in opinions:
        category = opinion['category']
        polarity = opinion['opinion polarity']
        if polarity != 'conflict':
            collected_opinions.append([category, [], polarity])
    return collected_opinions

In [22]:
total['sentence'] = total.sentence.apply(collect_sentences)
total['opinions'] = total.opinions.apply(collect_opinions)
total = total[total.opinions.apply(lambda x: x == []) == False]
total = total.reset_index()
total = total.set_axis(['id', 'sentence_form', 'annotation'], axis='columns')

In [23]:
model_checkpoint = 'snunlp/KR-ELECTRA-discriminator'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [24]:
total.sentence_form.iloc[0]

'잔차 사서 일주일쯤 지났다. 그동안 비가 와서 생각만큼 많이 타진 못했고 그래도 며칠은 새로 잔 자전거로 출퇴근. 첫날은 허벅지가 터질 거 같은 느낌이 들어서 이러다 비욘세 언냐를 뛰어넘어 뭐가 무서운 일이 벌어지는 게 아닐까 싶었지만 -_; 태생적인 한계로 인해 느낌만 그럴 뿐 아무 이상 없음. 둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악. 이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확 떨어졌는데 산 곳 가져가서 확인하니 기어 텐션 문제라고 고장 아니래.'

In [25]:
test = total.sentence_form.apply(lambda x: len(tokenizer.encode(x, '본품#품질', truncation=True))).iloc[0]
# tokenizer.decode(test)
test

151

In [26]:
test = [1,2,3,4,5,6,7,8,9]
test[:len(test)//2]

[1, 2, 3, 4]

In [27]:
total.sentence_form.iloc[0]
# total.annotation.iloc[0]

'잔차 사서 일주일쯤 지났다. 그동안 비가 와서 생각만큼 많이 타진 못했고 그래도 며칠은 새로 잔 자전거로 출퇴근. 첫날은 허벅지가 터질 거 같은 느낌이 들어서 이러다 비욘세 언냐를 뛰어넘어 뭐가 무서운 일이 벌어지는 게 아닐까 싶었지만 -_; 태생적인 한계로 인해 느낌만 그럴 뿐 아무 이상 없음. 둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악. 이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확 떨어졌는데 산 곳 가져가서 확인하니 기어 텐션 문제라고 고장 아니래.'

In [28]:
count_tags(total, entity_property_pair)

tags found:  1195
tag set of df:  20
tag set of offered:  25
difference:  {'제품 전체#다양성', '본품#가격', '패키지/구성품#가격', '브랜드#디자인', '본품#인지도'}
제품 전체#일반		290
본품#품질		178
본품#일반		148
제품 전체#품질		103
제품 전체#디자인	75
본품#편의성		58
제품 전체#인지도	58
제품 전체#편의성	56
브랜드#일반		48
패키지/구성품#디자인	43
제품 전체#가격		33
패키지/구성품#편의성	33
패키지/구성품#일반	19
본품#다양성		19
본품#디자인		9
패키지/구성품#품질	7
브랜드#품질		7
브랜드#인지도		7
브랜드#가격		3
패키지/구성품#다양성	1


In [29]:
DATA_V = 'uncleaned_v23'
save_path = f'./dataset/{DATA_V}'
print(save_path)

./dataset/uncleaned_v23


In [30]:
!mkdir -p {save_path}

total.to_json(f'{save_path}/absa2021.json', force_ascii=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
